In [10]:
import numpy as np
from typing import Callable, TypeVar
from collections import Counter, deque
import itertools
import regex as re
from intervaltree import Interval, IntervalTree


np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

processors = {
  'int_list': lambda x: [int(y) for y in x.split()]
}

def search(nodes, start, get_neighbors, end_condition=lambda _, __: False, dfs=True):
    q, visited = deque([(start, 0)]), {}
    while q:
        current, distance = q.popleft() if dfs else q.pop()
        if end_condition(current, distance):
            return visited, current
        if current in visited:
            continue
        for node in get_neighbors(current, distance):
            q.append((node, distance+1))
        visited[current] = distance
    return visited, None

In [7]:

def day1():
    loc1, loc2 = zip(*data(1, processors['int_list']))
    part1 = sum(abs(x[0]-x[1]) for x in zip(sorted(loc1), sorted(loc2)))
    counts = Counter(loc2)
    part2 = sum(x*counts[x] for x in loc1)
    return part1, part2

day1()

(1941353, 22539317)

In [80]:
def day2():
    def check_safe(report):
        ascending = sorted(report)
        diffs = np.diff(ascending)
        return max(diffs) <= 3 and min(diffs) >= 1 and (
            report == ascending or
            report == list(reversed(ascending))
        )

    def check_safe_damp(report):
        if check_safe(report):
            return 1, 1
        for damped in itertools.combinations(report, len(report)-1):
            if check_safe(list(damped)):
                return 0, 1
        return 0, 0

    reports = data(2, processors['int_list'])
    safe = np.array((0,0))
    for report in reports:
        safe += check_safe_damp(report)
    return safe

day2()

array([356, 413])

In [76]:
def day3():
    def mul_strings(s):
        x, y = s.split(',')
        return int(x)*int(y)

    instructions = ''.join(data(3))
    matches = list(re.finditer(r'mul\((\d+,\d+)\)', instructions))
    conds = list(re.finditer(r"don't\(\).+?do\(\)", instructions))
    donts = IntervalTree([Interval(*cond.span()) for cond in conds])
    result = sum([mul_strings(mul[1]) * (1 if not donts[mul.span()[0]] else 1j) for mul in matches])
    return int(result.real+result.imag), int(result.real)

day3()

(182780583, 90772405)

In [131]:
def day4():
    grid = np.array(data(4, lambda x: np.array(list(x))))
    target = 'XMAS'
    rev = target[::-1]
    ymax, xmax = grid.shape
    count = 0
    # print(grid)

    def get_neighbors(current, distance):
        col, row = int(current.real), int(current.imag)
        target_letter = target[distance]
        if grid[row, col] != target_letter:
            return
        if distance == len(target)-1:
            total.add(current)
            return
        for v in [1, -1, 1j, -1j, 1+1j, 1-1j, -1+1j, -1-1j]:
            new = current + v
            x, y = int(new.real), int(new.imag)
            if not (y >= 0 and x >= 0 and y < ymax and x < xmax):
                continue
            yield new
    
    for j in range(ymax):
        for i in range(xmax):
            # Find diagonals - already implemented from misreading the question
            total = set()
            coordinate = i+1j*j
            search(grid, coordinate, get_neighbors)
            for x in total:
                distance = x-coordinate
                if abs(distance.real) == 3 and abs(distance.imag) == 3:
                    count += 1
                elif distance.real == 3 and not distance.imag:
                    if ''.join(grid[j, i:i+4]) == target:
                        # print('a', x, coordinate, distance, i, j, grid[j, i:i+4])
                        count += 1
                elif distance.real == -3 and not distance.imag:
                    # print('b', x, coordinate, distance, i, j, grid[j, i-3:i+1], rev)
                    if ''.join(grid[j, i-3:i+1]) == rev:
                        count += 1
                elif not distance.real and distance.imag == 3:
                    if ''.join(grid[j:j+4, i]) == target:
                        count += 1
                elif not distance.real and distance.imag == -3:
                    if ''.join(grid[j-3:j+1, i]) == rev:
                        count += 1

    return count

day4()

2599

In [125]:
'test'[::-1]

'tset'